In [18]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager

# Import pandas dependencies
import pandas as pd

In [19]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/ryleejensen/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache


In [20]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [21]:
# Set up the html parser
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [22]:
# Find the title to the article
slide_elem.find('div', class_='content_title')

<div class="content_title">NASA's Curiosity Takes Selfie With 'Mary Anning' on the Red Planet</div>

In [23]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"NASA's Curiosity Takes Selfie With 'Mary Anning' on the Red Planet"

In [24]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'The Mars rover has drilled three samples of rock in this clay-enriched region since arriving in July.'

### Featured Images

In [25]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [26]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [27]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [28]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars3.jpg'

In [29]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

In [30]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [31]:
# Convert DataFrame back into HTML-ready code
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

### Scrape High-Resolution Mars' Hemisphere Images and Titles

In [51]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'

browser.visit(url)

In [52]:
# 2. Create a list to hold the images and titles.
hemisphere_image_url = []

# 3. Write code to retrieve the image urls and titles for each hemisphere.
hemi_html = browser.html
hemi_soup = soup(hemi_html, 'html.parser')

items = hemi_soup.find_all('div', class_='item')

In [53]:
# Create a loop to scrape through all the hemisphere information.
for x in items:
    hemisphere = {}
    titles = x.find('h3').text
    
    # Create a link for the image urls
    link_ref = x.find('a', class_='itemLink product-item')['href']
    browser.visit(url + link_ref)
    
    # Parse the data
    image_html = browser.html
    image_soup = soup(image_html, 'html.parser')
    download = image_soup.find('div', class_= 'downloads')
    img_url = (url + link_ref)
    
    print(titles)
    print(img_url)
    
    # Append list
    hemisphere['img_url'] = img_url
    hemisphere['title'] = titles
    hemisphere_image_url.append(hemisphere)
    browser.back()

Cerberus Hemisphere Enhanced
https://marshemispheres.com/cerberus.html
Schiaparelli Hemisphere Enhanced
https://marshemispheres.com/schiaparelli.html
Syrtis Major Hemisphere Enhanced
https://marshemispheres.com/syrtis.html
Valles Marineris Hemisphere Enhanced
https://marshemispheres.com/valles.html


In [54]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_url

[{'img_url': 'https://marshemispheres.com/cerberus.html',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/schiaparelli.html',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/syrtis.html',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/valles.html',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [55]:
# 5. Quit the browser
browser.quit()